In [1]:
import csv
import copy
import argparse
import itertools

from collections import Counter
from collections import deque

import os
import cv2 as cv
import numpy as np
import mediapipe as mp

In [2]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.9,
    min_tracking_confidence=0.8,
)

In [8]:
path = r'D:\Kuliah\Tugas, PPT, Buku Kuliah\Semester 5\Data Mining II\Project\Image Augmentation'
path_sibi = r'D:\Kuliah\Tugas, PPT, Buku Kuliah\Semester 5\Data Mining II\Project\SIBI'
path_a = r"D:\Kuliah\Tugas, PPT, Buku Kuliah\Semester 5\Data Mining II\Project\A"

In [10]:
abjad = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
for i in range(0,26):
    path_abjad = f"{path}\{abjad[i]}"
    for gambar in os.listdir(path_abjad):
        # Read Image
        image = cv.imread(f"{path_abjad}\{gambar}")
        image = cv.flip(image, 1)  # Mirror display
        debug_image = copy.deepcopy(image)
        # Detection implementation #############################################################
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        #  ####################################################################
        if results.multi_hand_landmarks is not None:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                                    results.multi_handedness):
                # Landmark calculation
                landmark_list = calc_landmark_list(debug_image, hand_landmarks)

                # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(
                    landmark_list)

                # Write to the dataset file
                logging_csv(i, pre_processed_landmark_list)
    print("Abjad",abjad[i],"Selesai!")


Abjad A Selesai!
Abjad B Selesai!
Abjad C Selesai!
Abjad D Selesai!
Abjad E Selesai!
Abjad F Selesai!
Abjad G Selesai!
Abjad H Selesai!
Abjad I Selesai!
Abjad J Selesai!
Abjad K Selesai!
Abjad L Selesai!
Abjad M Selesai!
Abjad N Selesai!
Abjad O Selesai!
Abjad P Selesai!
Abjad Q Selesai!
Abjad R Selesai!
Abjad S Selesai!
Abjad T Selesai!
Abjad U Selesai!
Abjad V Selesai!
Abjad W Selesai!
Abjad X Selesai!
Abjad Y Selesai!
Abjad Z Selesai!


In [55]:
image = cv.imread(r"D:\Kuliah\Tugas, PPT, Buku Kuliah\Semester 5\Data Mining II\Project\Code\mediapipe\IMG20221130164738.jpg")
image

array([[[143, 179, 203],
        [142, 178, 202],
        [142, 178, 202],
        ...,
        [179, 207, 231],
        [179, 206, 232],
        [180, 207, 233]],

       [[142, 178, 202],
        [141, 177, 201],
        [141, 177, 201],
        ...,
        [179, 207, 231],
        [179, 206, 232],
        [179, 206, 232]],

       [[140, 176, 200],
        [139, 175, 199],
        [140, 176, 200],
        ...,
        [180, 208, 232],
        [179, 207, 231],
        [179, 207, 231]],

       ...,

       [[ 49,  73, 119],
        [ 46,  70, 116],
        [ 46,  70, 116],
        ...,
        [102, 135, 161],
        [101, 134, 160],
        [100, 133, 159]],

       [[ 49,  73, 119],
        [ 45,  69, 115],
        [ 44,  68, 114],
        ...,
        [102, 135, 161],
        [101, 134, 160],
        [100, 133, 159]],

       [[ 49,  73, 119],
        [ 45,  69, 115],
        [ 45,  69, 115],
        ...,
        [101, 134, 160],
        [101, 134, 160],
        [100, 133, 159]]

In [56]:

# Camera capture #####################################################

# ret, image = cap.read()
# if not ret:
#     break
# image = cv.flip(image, 1)  # Mirror display
debug_image = copy.deepcopy(image)

# Detection implementation #############################################################
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
image.flags.writeable = False
results = hands.process(image)
image.flags.writeable = True

In [4]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

def logging_csv(number, landmark_list):
    csv_path = 'model/keypoint_classifier/keypoint.csv'
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f)
        writer.writerow([number, *landmark_list])

In [57]:
#  ####################################################################
# if results.multi_hand_landmarks is not None:
for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                        results.multi_handedness):
    # Landmark calculation
    landmark_list = calc_landmark_list(debug_image, hand_landmarks)

    # Conversion to relative coordinates / normalized coordinates
    pre_processed_landmark_list = pre_process_landmark(
        landmark_list)
    print(pre_processed_landmark_list)

        # # Write to the dataset file
        # logging_csv(0, pre_processed_landmark_list)


[0.0, 0.0, 0.37582865884752675, 0.021417644059153494, 0.5956144824069353, -0.06425293217746048, 0.7883732789393166, -0.07802141764405915, 0.9964303926568078, -0.1172870984191739, 0.4604793472718001, -0.4961754207037226, 0.6455889852116268, -0.6211116777154513, 0.8322284548699643, -0.5961244263131056, 0.9760326364099949, -0.5425803161652218, 0.41866394696583376, -0.5645079041305456, 0.6568077511473738, -0.7332993370729219, 0.8704742478327384, -0.6955634880163182, 1.0, -0.5752167261601224, 0.37735849056603776, -0.6160122386537481, 0.6114227434982152, -0.791942886282509, 0.8245792962774094, -0.7526772055073941, 0.9643039265680775, -0.660377358490566, 0.34625191228964813, -0.6262111167771545, 0.5711371749107598, -0.7899031106578276, 0.7450280469148394, -0.7975522692503825, 0.8842427332993371, -0.76185619581846]
